In [10]:
import os
HOME = os.getcwd()

%cd {HOME}

import sys
!{sys.executable} -m pip install 'git+https://github.com/facebookresearch/segment-anything.git'

!pip install -q jupyter_bbox_widget roboflow dataclasses-json supervision

%cd {HOME}
!mkdir {HOME}/weights
%cd {HOME}/weights

!wget -q https://dl.fbaipublicfiles.com/segment_anything/sam_vit_h_4b8939.pth


CHECKPOINT_PATH = os.path.join(HOME, "weights", "sam_vit_h_4b8939.pth")
print(CHECKPOINT_PATH, "; exist:", os.path.isfile(CHECKPOINT_PATH))



/content/weights
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/facebookresearch/segment-anything.git to /tmp/pip-req-build-8plb2prn
  Running command git clone --filter=blob:none --quiet https://github.com/facebookresearch/segment-anything.git /tmp/pip-req-build-8plb2prn
  Resolved https://github.com/facebookresearch/segment-anything.git to commit 6fdee8f2727f4506cfbbe553e23b895e27956588
  Preparing metadata (setup.py) ... done
/content/weights
/content/weights/weights
/content/weights/weights/sam_vit_h_4b8939.pth ; exist: True


In [11]:
# Load the Drive helper and mount
from google.colab import drive


# This will prompt for authorization.
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [12]:
import cv2
import supervision as sv
from segment_anything import sam_model_registry, SamAutomaticMaskGenerator, SamPredictor
import torch

In [13]:
DEVICE = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
MODEL_TYPE = "vit_h"

sam = sam_model_registry[MODEL_TYPE](checkpoint=CHECKPOINT_PATH).to(device=DEVICE)

In [14]:
mask_generator = SamAutomaticMaskGenerator(sam)

## Questa sezione serve per non processare immagini di cui si ha già la maschera

In [19]:
dir_path_masks = "/content/drive/MyDrive/Colab_Notebooks/Machine_Learning/Progetto/Dataset/Masks/"
name_masks = os.listdir(dir_path_masks)
dir_path = "/content/drive/MyDrive/Colab_Notebooks/Machine_Learning/Progetto/Dataset/Images/"
files = os.listdir(dir_path)


print("Numero di Immagini ->", len(files))
print(files)
print("Numero di Mask ->", len(name_masks))
print(name_masks)

name_masks2 = []
for i in range(len(name_masks)):
  name_masks2.append(name_masks[i].replace("_mask.png", "")) 

files2 = []
for i in range(len(files)):
  files2.append(files[i].replace(".jpg", ""))

# Converte le liste in set
set1 = set(name_masks2)
set2 = set(files2)
# Trova gli elementi comuni
elementi_comuni = set1.intersection(set2)

# Converti gli elementi comuni in una lista
elementi_comuni_lista = list(elementi_comuni)

# Rimuovi gli elementi comuni dalla lista2
for elemento in elementi_comuni_lista:
    elemento = elemento + ".jpg"
    files.remove(elemento)

# Stampa le liste risultanti

num_image = len(files)

Numero di Immagini -> 612
['10cent_1.jpg', '10cent_2.jpg', '10cent_3.jpg', '10cent_4.jpg', '10cent_5.jpg', '10cent_6.jpg', '10cent_7.jpg', '10cent_8.jpg', '10cent_9.jpg', '10cent_10.jpg', '10cent_11.jpg', '10cent_12.jpg', '10cent_13.jpg', '10cent_14.jpg', '10cent_15.jpg', '10cent_16.jpg', '10cent_17.jpg', '10cent_18.jpg', '10cent_19.jpg', '10cent_20.jpg', '10cent_21.jpg', '10cent_22.jpg', '10cent_23.jpg', '10cent_24.jpg', '10cent_25.jpg', '10cent_26.jpg', '10cent_27.jpg', '10cent_28.jpg', '10cent_29.jpg', '10cent_30.jpg', '10cent_31.jpg', '10cent_32.jpg', '10cent_33.jpg', '10cent_34.jpg', '10cent_35.jpg', '10cent_36.jpg', '10cent_37.jpg', '10cent_38.jpg', '10cent_39.jpg', '10cent_40.jpg', '10cent_41.jpg', '10cent_42.jpg', '10cent_43.jpg', '10cent_44.jpg', '10cent_45.jpg', '10cent_46.jpg', '10cent_47.jpg', '10cent_48.jpg', '10cent_49.jpg', '10cent_50.jpg', '10cent_51.jpg', '10cent_52.jpg', '10cent_53.jpg', '10cent_54.jpg', '10cent_55.jpg', '10cent_56.jpg', '10cent_57.jpg', '10cent_58.jp

In [20]:
num_image

541

# Sistemare
Può accadere, e accadrà, che la soluzione trovata _prendere la maschera con l'area più grande_ non funzioni, inoltre biosgna capire come prendere le maschere di più moenete per creare una sola maschera qual'ora nell'immagien siano presenti più monete da estrarre.

# Fixato


In [ ]:
for i in range(num_image):
  IMAGE_NAME = files[i]
  IMAGE_PATH = dir_path + IMAGE_NAME
  #print(IMAGE_NAME)
  #print(IMAGE_PATH)
  print("Elaborazione Immagine -> ", IMAGE_NAME)


  image_bgr = cv2.imread(IMAGE_PATH)
  image_rgb = cv2.cvtColor(image_bgr, cv2.COLOR_BGR2RGB)
  print("Calcolo Maschere")
  sam_result = mask_generator.generate(image_rgb)

  new_masks = sorted(sam_result, key=lambda x: x['area'], reverse=True)
  
  if new_masks[0]["bbox"][:2]== [0,0]:  # Questo if Serve per provare a risolvere 
    new_masks.pop(0)                    # il problema della maschera di background
    print("Maschera cancellata dalla foto ------->", IMAGE_NAME)

  img_np = (new_masks[0]['segmentation'])
  #print(type(img_np))
  img_tt = torch.from_numpy(img_np)
  #print(type(img_tt))
  #print(img_tt)

  #print(img_tt.shape)
  img_tt = img_tt[None, : ]
  #print(img_tt.shape)
  #print(img_tt)

  img = img_tt.long().type(torch.uint8)

  MASK_NAME = IMAGE_NAME[:-4] + "_mask" + IMAGE_NAME[-4] + "png"
  MASK_PATH = dir_path.replace("Images", "Masks", 1) + MASK_NAME
  #print(MASK_NAME)
  #print(MASK_PATH)

  # Trasponi il tensore (riorganizza le dimensioni)
  tensor = img.permute(1, 2, 0)
  # Converte il tensore in un array NumPy
  array = tensor.numpy()
  print("Salvataggio maschera #", i + 1)
  cv2.imwrite(MASK_PATH, array)

Elaborazione Immagine ->  10cent_73.jpg
Calcolo Maschere
Salvataggio maschera # 1
Elaborazione Immagine ->  10cent_74.jpg
Calcolo Maschere
Salvataggio maschera # 2
Elaborazione Immagine ->  10cent_75.jpg
Calcolo Maschere
Salvataggio maschera # 3
Elaborazione Immagine ->  10cent_76.jpg
Calcolo Maschere
Salvataggio maschera # 4
Elaborazione Immagine ->  10cent_77.jpg
Calcolo Maschere
Salvataggio maschera # 5
Elaborazione Immagine ->  10cent_78.jpg
Calcolo Maschere
Salvataggio maschera # 6
Elaborazione Immagine ->  10cent_79.jpg
Calcolo Maschere
Salvataggio maschera # 7
Elaborazione Immagine ->  10cent_80.jpg
Calcolo Maschere
Salvataggio maschera # 8
Elaborazione Immagine ->  10cent_81.jpg
Calcolo Maschere
Salvataggio maschera # 9
Elaborazione Immagine ->  10cent_82.jpg
Calcolo Maschere
Salvataggio maschera # 10
Elaborazione Immagine ->  10cent_83.jpg
Calcolo Maschere
Salvataggio maschera # 11
Elaborazione Immagine ->  10cent_84.jpg
Calcolo Maschere
Salvataggio maschera # 12
Elaborazione 